In [23]:
import requests
from IPython.display import display, HTML
page = 7
r = requests.get(f"http://api.alquran.cloud/v1/page/{page}/quran-uthmani")
d = r.json()["data"]
a = []
for ayah in d["ayahs"]:
    a.append(f"{ayah['text']} <span style='color:#d4af37;font-size:.8em;margin-left:12px'>{ayah['numberInSurah']}</span>")
html = f"""
<div style="direction:rtl;text-align:right;font-family:'Amiri','Scheherazade New','Times New Roman',serif;font-size:28px;line-height:2.4">
{ ' '.join(a) }
</div>
"""
display(HTML(html))


In [24]:
import requests
from IPython.display import display, HTML
page = 7
ar = requests.get(f"http://api.alquran.cloud/v1/page/{page}/quran-uthmani").json()["data"]["ayahs"]
en = requests.get(f"http://api.alquran.cloud/v1/page/{page}/en.sahih").json()["data"]["ayahs"]
s = []
for a,e in zip(ar,en):
    s.append(f"<div style='margin:10px 0'><div style=\"direction:rtl;text-align:right;font-family:'Amiri','Scheherazade New','Times New Roman',serif;font-size:24px;line-height:2.2\">{a['text']} <span style='color:#d4af37;font-size:.8em;margin-left:10px'>{a['numberInSurah']}</span></div><div style=\"font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:16px;line-height:1.6;color:#eee\">{e['text']}</div></div>")
html = f"""
<div style="background:#1e293b;padding:16px;border-radius:10px">{''.join(s)}</div>
"""
display(HTML(html))


In [25]:
import base64
from IPython.display import HTML, Javascript

def save_as_image(html_content, filename):
    js_code = f"""
    var element = document.createElement('div');
    element.innerHTML = `{html_content}`;
    element.style.position = 'absolute';
    element.style.left = '-9999px';
    element.style.top = '-9999px';
    document.body.appendChild(element);
    
    html2canvas(element).then(canvas => {{
        var link = document.createElement('a');
        link.download = '{filename}';
        link.href = canvas.toDataURL();
        link.click();
        document.body.removeChild(element);
    }});
    """
    
    display(Javascript(js_code))

def save_page_as_image(page_num):
    ar = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/quran-uthmani").json()["data"]["ayahs"]
    en = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/en.sahih").json()["data"]["ayahs"]
    
    s = []
    for a,e in zip(ar,en):
        s.append(f"<div style='margin:10px 0'><div style=\"direction:rtl;text-align:right;font-family:'Amiri','Scheherazade New','Times New Roman',serif;font-size:24px;line-height:2.2\">{a['text']} <span style='color:#d4af37;font-size:.8em;margin-left:10px'>{a['numberInSurah']}</span></div><div style=\"font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:16px;line-height:1.6;color:#eee\">{e['text']}</div></div>")
    
    html = f"""
    <div style="background:#1e293b;padding:16px;border-radius:10px;width:800px">{''.join(s)}</div>
    """
    
    save_as_image(html, f"quran_page_{page_num}.png")

save_page_as_image(7)


<IPython.core.display.Javascript object>

In [26]:
import os
import subprocess
import tempfile

def save_page_as_image(page_num):
    ar = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/quran-uthmani").json()["data"]["ayahs"]
    en = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/en.sahih").json()["data"]["ayahs"]
    
    s = []
    for a,e in zip(ar,en):
        s.append(f"<div style='margin:10px 0'><div style=\"direction:rtl;text-align:right;font-family:'Amiri','Scheherazade New','Times New Roman',serif;font-size:24px;line-height:2.2\">{a['text']} <span style='color:#d4af37;font-size:.8em;margin-left:10px'>{a['numberInSurah']}</span></div><div style=\"font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:16px;line-height:1.6;color:#eee\">{e['text']}</div></div>")
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <link href="https://fonts.googleapis.com/css2?family=Amiri:wght@400;700&family=Scheherazade+New:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {{ margin: 0; padding: 20px; background: #1e293b; color: white; }}
        </style>
    </head>
    <body>
        <div style="background:#1e293b;padding:16px;border-radius:10px;width:800px">{''.join(s)}</div>
    </body>
    </html>
    """
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='.html', delete=False, encoding='utf-8') as f:
        f.write(html_content)
        html_file = f.name
    
    output_file = f"quran_page_{page_num}.png"
    
    try:
        subprocess.run([
            "wkhtmltoimage", 
            "--width", "1000",
            "--height", "1400",
            "--format", "png",
            html_file, 
            output_file
        ], check=True)
        print(f"✅ Image saved as {output_file}")
        os.unlink(html_file)
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("❌ wkhtmltoimage not found. Installing...")
        try:
            subprocess.run(["brew", "install", "wkhtmltopdf"], check=True)
            subprocess.run([
                "wkhtmltoimage", 
                "--width", "1000",
                "--height", "1400", 
                "--format", "png",
                html_file, 
                output_file
            ], check=True)
            print(f"✅ Image saved as {output_file}")
            os.unlink(html_file)
        except:
            print("❌ Could not install wkhtmltoimage. Please install manually:")
            print("brew install wkhtmltopdf")
            print(f"Then run: wkhtmltoimage {html_file} {output_file}")

save_page_as_image(7)


❌ wkhtmltoimage not found. Installing...


==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
addlicense: Scan directories recursively to ensure source files have license headers
anchor: Solana Program Framework
bom: Utility to generate SPDX-compliant Bill of Materials manifests
changelogen: Generate Beautiful Changelogs using Conventional Commits
claude-cmd: Claude Code Commands Manager
claude-hooks: Hook system for Claude Code
codebook-lsp: Code-aware spell checker language server
context7-mcp: Up-to-date code documentation for LLMs and AI code editors
docker-compose-langserver: Language service for Docker Compose documents
dqlite: Embeddable, replicated and fault-tolerant SQLite-powered engine
dstp: Run common networking tests against your site
gitingest: Turn any Git repository i

❌ Could not install wkhtmltoimage. Please install manually:
brew install wkhtmltopdf
Then run: wkhtmltoimage /var/folders/bk/50ynwg0s2vv4khkm404pgghc0000gn/T/tmp5ltu_n72.html quran_page_7.png


Error: Cask 'wkhtmltopdf' has been disabled because it is discontinued upstream! It was disabled on 2024-12-16.
